In [2]:
# !pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 965.9 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 2.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 6.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 1.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 41.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.7 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 4.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 5.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 18.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 45.9 MB/s eta 0:00:00:0

In [1]:
from pycaret.regression import *
import pandas as pd
import numpy as np

# Load data from CSV
file_path = '/home/jovyan/shared-public/ml_swe_monitoring_prod/snotel_ghcnd_stations_4yrs_all_cols_small.csv'  # Replace 'your_dataset.csv' with the path to your CSV file

# this file ^ is a random 10,000 rows for the big one 

# SNOTEL is the ground truth for SWE (swe_value column) 
# all other variables (e.g. slope, aspect, elevation, met...) are inputs 
# if given these variables, and no ground truth, can the model give us a SWE estimate? 

# Read the first 10,000 rows of the CSV file
data = pd.read_csv(file_path)

# # Display the first few rows to verify
# print(data.head())
# data

# Remove rows where 'swe_value' is -999.0
data = data[data['swe_value'] != -999.0]

# Display the first few rows after removing invalid values
print("\nData after removing -999.0 values:")
print(data.head())

# Optionally, save the cleaned data to a new CSV file
cleaned_file_path = '/home/jovyan/shared-public/ml_swe_monitoring_prod/snotel_ghcnd_stations_4yrs_all_cols_small_cleaned.csv'
data.to_csv(cleaned_file_path, index=False)

print(f"\nCleaned data saved to {cleaned_file_path}")


Data after removing -999.0 values:
         date       lat        lon  SWE station_name  swe_value  \
0  2019-11-06  48.50910 -110.96940  0.0  USC00241692        0.0   
2  2021-04-19  33.60106 -111.71285  0.0  USC00023190        0.0   
3  2019-12-19  38.25000 -120.85000  0.0            0        0.0   
4  2019-08-31  41.37200 -122.23000  0.0            0        0.0   
5  2021-03-06  37.42001 -105.05278  0.0            0        0.0   

   change_in_swe_inch  snow_depth  air_temperature_observed_f  \
0                 0.0         0.0                         0.0   
2                 0.0         0.0                         0.0   
3                 0.0         0.0                         0.0   
4                 0.0         0.0                         0.0   
5                 0.0         0.0                         0.0   

   precipitation_amount  ...  cumulative_SWE  cumulative_air_temperature_tmmn  \
0                   0.0  ...       -1.000000                        -1.000000   
2       

In [2]:
# Shuffle and select 10% of the data for training
data_shuffled = data.sample(frac=1, random_state=42)  # Shuffle the data
data_train = data_shuffled.sample(frac=0.10, random_state=42)  # Select 10% of the data

# Initialize PyCaret setup
exp_reg = setup(data, target='swe_value', session_id=42) # target is the column we want to predict 

# Compare models
best_model = compare_models()

# Print the best model
print(best_model)

,Description,Value
0,Session id,42
1,Target,swe_value
2,Target type,Regression
3,Original data shape,"(6774, 105)"
4,Transformed data shape,"(6774, 105)"
5,Transformed train set shape,"(4741, 105)"
6,Transformed test set shape,"(2033, 105)"
7,Numeric features,102
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,0.4678,7.0088,2.4900,0.8003,0.2145,0.4532,4.8620
et,Extra Trees Regressor,0.5246,8.6974,2.7852,0.7520,0.2194,0.5376,1.4060
lightgbm,Light Gradient Boosting Machine,0.5688,8.6089,2.8044,0.7516,0.2461,0.4579,72.5470
gbr,Gradient Boosting Regressor,0.6114,8.5276,2.7805,0.7512,0.2665,0.6565,2.5700
xgboost,Extreme Gradient Boosting,0.5617,8.7810,2.8113,0.7498,0.2202,0.4995,0.8990
en,Elastic Net,0.9124,9.3391,2.8679,0.7312,0.3980,0.9883,0.2180
llar,Lasso Least Angle Regression,0.8445,9.4057,2.8773,0.7292,0.3494,0.9561,0.0630
lasso,Lasso Regression,0.8445,9.4055,2.8772,0.7292,0.3494,0.9561,0.1280
omp,Orthogonal Matching Pursuit,0.8625,9.4571,2.8863,0.7274,0.3799,1.0221,0.0630
dt,Decision Tree Regressor,0.5808,9.9398,2.9828,0.7142,0.2779,0.5713,0.1460


RandomForestRegressor(n_jobs=-1, random_state=42)


In [ ]:
# Compare models using Mean Absolute Error (MAE) as the evaluation metric
best_model = compare_models(sort='MAE')

# Tune the hyperparameters of the best model
tuned_model = tune_model(best_model)

# Evaluate the tuned model using cross-validation
evaluate_model(tuned_model)

# Plot residuals to assess the model's performance on extreme values
plot_model(tuned_model, plot='residuals')

In [ ]:
# Visualize correlation matrix
plot_model(type='correlation')

# Visualize distribution of target variable
plot_model(type='distribution')

# Visualize feature importance
plot_model(tuned_model, plot='feature')

# Visualize residuals
plot_model(tuned_model, plot='residuals')

# Visualize learning curves
plot_model(tuned_model, plot='learning')